In [1]:
import pandas as pd
import os
import numpy as np
import tqdm
import math

In [2]:
folder = '../big_data/'
MAX_ITER = 7
no_embedings = []

# Functions

In [3]:
def get_embeding(word,emb):
    return emb.loc[word]

In [4]:
def create_tags_list_dict(row):
    # dictionary keyword: list of concepts
    result = {}
    for tag in row:
        if tag[0] in result.keys():
            if  tag[1] not in result[tag[0]]:
                result[tag[0]].append(tag[1])
        else:
            result[tag[0]]= [tag[1]]


    return result

In [5]:
def disambiguation(current_selection,embedings):
    ''' 
    current_selection : dictionary keyword: list of all unique concepts
    
    '''
    # we iterate over the current_selection MAX_ITER times
    for i in range(MAX_ITER):
        for keyword, concepts_list in current_selection.items():
            distances = {} # for each possible concept calaculate the mean distance from other kewords (concepts of them)
            for concept in concepts_list:
                distances[concept] = []
                for k, current_best_tags in current_selection.items():
                    # foreach keyword that is not a current one 
                    if k!=keyword:
                        current_best_tag = current_best_tags[0] # the first out of list of concepts
                        try:
                            distances[concept].append(math.dist(get_embeding(concept,embedings),get_embeding(current_best_tag,embedings))) # append distance from this concept
                        except Exception as e:
                            print(e)
                distances[concept] = np.mean(distances[concept]) # mean distance 
            current_selection[keyword] = sorted(distances, key=distances.get) # upadate the current selection of this keyword
    return current_selection
    

# Data

## Train

In [6]:
train = pd.read_csv(os.path.join(folder,'bertopic_ncbo_fulldata_lemmatize_nostopwords_data_2022-11-30_12-09-18_train.csv'))
train['ncbo_annotations_pairs'] = train['ncbo_annotations_pairs'].apply(eval)

In [10]:

embedings_fine_tuned = pd.read_csv(os.path.join(folder,'bertopic_ncbo_fulldata_lemmatize_nostopwords_data_2022-11-30_12-09-18_train_model_fine_tuned.csv'))
embedings_fine_tuned = embedings_fine_tuned.set_index('words')

embedings_fine_pretrained = pd.read_csv(os.path.join(folder,'bertopic_ncbo_fulldata_lemmatize_nostopwords_data_2022-11-30_12-09-18_train_model_pretrained.csv'))
embedings_fine_pretrained = embedings_fine_pretrained.set_index('words')

In [ ]:
embedings_fine_tuned.head(3)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
words,,,,,,,,,,,,,,,,,,,,,
Metabolic Process,0.583000,-0.012395,-0.401520,-0.376704,0.660073,0.438554,0.571691,0.231643,-0.742332,-0.904984,...,0.450005,-0.112175,0.501938,0.195895,0.017445,0.118989,-0.311507,-0.254814,-0.192918,-0.199149
Obesity [Disease/Finding],-0.123698,-0.108045,0.094596,-0.415700,0.554280,0.058951,0.278378,0.431898,-0.240664,-0.126073,...,0.136707,0.001791,0.070385,-0.229363,0.334851,0.287971,0.090172,-0.100504,-0.173628,-0.362540
PVC group,0.666851,-0.378545,-0.242242,0.146456,0.470777,0.347311,-0.229048,0.309446,-0.451586,0.247780,...,-0.002421,0.376668,0.559750,0.034041,0.508923,0.197937,-0.209565,0.010399,-0.212529,-0.341406


In [ ]:
possible_tags = train['ncbo_annotations_pairs'].apply(lambda r: create_tags_list_dict(r))

In [ ]:
possible_tags

0       {'CELL': ['Diagnostic, Therapeutic, or Researc...
1       {'CELL': ['Diagnostic, Therapeutic, or Researc...
2       {'CELL': ['Diagnostic, Therapeutic, or Researc...
3       {'SPINE': ['Anatomical entity', 'Musculoskelet...
4       {'PATIENT': ['Veterinary Patient', 'Person', '...
                              ...                        
3508    {'CELL': ['Diagnostic, Therapeutic, or Researc...
3509    {'FOOD': ['Physical Object', 'Substance', 'Foo...
3510    {'ECL': ['Clinical Pathology Procedure', 'Cyto...
3511    {'ASTHMA': ['Asthma Pathway', 'Asthma', 'Lung ...
3512    {'IMPLANT': ['Unit by Category', 'Implant Dosi...
Name: ncbo_annotations_pairs, Length: 3513, dtype: object

In [ ]:
res = []
for current_selection in tqdm.tqdm(possible_tags):
    res.append(disambiguation(current_selection, embedings_fine_tuned))
train['after_disambiguation_fine_tuned'] = res

  4%|▎         | 131/3513 [04:28<2:28:05,  2.63s/it]c:\Users\gosia\anaconda3\envs\envNNN\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\gosia\anaconda3\envs\envNNN\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 3513/3513 [3:17:26<00:00,  3.37s/it]    


In [ ]:
res = []
for current_selection in tqdm.tqdm(possible_tags):
    res.append(disambiguation(current_selection, embedings_fine_pretrained))
train['after_disambiguation_pretrained'] = res

In [ ]:
train.to_csv(os.path.join(folder,'bertopic_ncbo_fulldata_lemmatize_nostopwords_data_2022-11-30_12-09-18_train_disambiguation.csv'))

## Test

In [ ]:
test = pd.read_csv(os.path.join(folder,'bertopic_ncbo_fulldata_lemmatize_nostopwords_data_2022-11-30_12-09-18_test.csv'))
test['ncbo_annotations_pairs'] = test['ncbo_annotations_pairs'].apply(eval)

In [ ]:

embedings_fine_tuned = pd.read_csv(os.path.join(folder,'bertopic_ncbo__fulldata_lemmatize_nostopwords_data_2022-11-30_12-09-18_test_model_fine_tuned.csv'))
embedings_fine_tuned = embedings_fine_tuned.set_index('words')

embedings_fine_pretrained = pd.read_csv(os.path.join(folder,'bertopic_ncbo__fulldata_lemmatize_nostopwords_data_2022-11-30_12-09-18_test_model_pretrained.csv'))
embedings_fine_pretrained = embedings_fine_pretrained.set_index('words')

In [ ]:
possible_tags = test['ncbo_annotations_pairs'].apply(lambda r: create_tags_list_dict(r))

In [ ]:
res = []
for current_selection in tqdm.tqdm(possible_tags):
    res.append(disambiguation(current_selection, embedings_fine_tuned))
test['after_disambiguation_fine_tuned'] = res

In [ ]:
res = []
for current_selection in tqdm.tqdm(possible_tags):
    res.append(disambiguation(current_selection, embedings_fine_pretrained))
test['after_disambiguation_pretrained'] = res

In [ ]:
test.to_csv(os.path.join(folder,'bertopic_ncbo_fulldata_lemmatize_nostopwords_data_2022-11-30_12-09-18_test_disambiguation.csv'))